# Average photon energy

In this tutorial you will calculate the average photon energy (APE) of different spectral irradiance distributions. The APE parameter is a useful indicator of the overall shape of the solar spectrum. Higher (lower) APE values indicate a blue (red) shift in the spectrum and is one of a variety of such characterisation methods that is used in the PV performance literature.

The first step is to import a few Python packages.

In [ ]:
# Install pvlib on Google Colab as this is not a standard package.
!pip install pvlib
!pip install scipy

In [ ]:
import pvlib  # library for PV and solar calculations
import pandas as pd  # library for data analysis
import matplotlib.pyplot as plt  # library for plotting
import numpy as np  # library for math and linear algebra
import scipy  # library used for scientific computing

## Step 1: Define parameters

The first step is to define a location and some meteorological parameters that we will use in the next step in order to simulate some spectra.

In [1]:
lat, lon = 55.7906, 12.5251  # DTU building 119
tilt = 34  # assume a tilt for calculating POA irradiance
azimuth = 180  # south-facing system
pressure = 100700  # at 50 m AMSL, roughly [Pa]
water_vapor_content = 0.5  # cm
tau500 = 0.1 # turbidity (AOD) at 500 nm
ozone = 0.31  # atmospheric ozone content atm-cm
albedo = 0.2  # ground surface albedo

## Step 2: Simulate some spectra

You can model spectral irradiance using [``pvlib.spectrum.spectrl2``](https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.spectrum.spectrl2.html). Note that since you are modeling spectra for more than one set of conditions (you will get 10 spectra, according to the ``times`` parameter) the function will return a dictionary containing 2-D arrays for the spectral irradiance components and a 1-D array of shape (122,) for wavelength. For each of the 2-D arrays, one dimension is for wavelength in nm and one is for irradiance in Wm⁻²nm⁻¹.

In [ ]:
times = pd.date_range('2023-03-01 08:00', freq='h', periods=10, tz='Europe/Copenhagen')

# Write your code here
solpos = 
aoi = 
relative_airmass = 
spectra_components = 

## Step 3: Visulaize the spectral data

Plot the ``poa_global`` as a function of the ``wavelength`` for the different spectra you have modeled.

In [ ]:
# Write your code here
time_labels = times.strftime("%H%M")  # use this as legend in your plot in order to be easier to identify the different times of the day
spectral_poa =
wavelength =


## Step 4: Normalize and plot the spectral irradiance

Given the changing broadband irradiance throughout the day, it is not obvious from the previous plot how the relative distribution of light changes as a function of wavelength. It is also hard to figure out if there is any shift in the spectrum at different times of the day. We can normalize the spectral irradiance curves to visualize this shift in the shape of the spectrum over the course of the day. In this example, we normalize by dividing each spectral irradiance value by the total broadband irradiance.

The total broadband irradiance can be calculated by integrating the entire spectral irradiance distribution (``poa_global``) with respect to ``wavelength``. The integral can be calculated using the [``scipy.integrate.trapezoid``](https://docs.scipy.org/doc/scipy-1.16.2/reference/generated/scipy.integrate.trapezoid.html):

In [ ]:
# Write your code here

broadband_irradiance =
spectral_poa_normalised = 


What do you notice about the normalized spectra?

## Step 5: Calculate the average photon energy (APE)

The amount of red and blue shifts that you notice in the figure above can be quantified by calculating the average photon energy (APE). APE can be calculated using [``pvlib.spectrum.average_photon_energy``](https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.spectrum.average_photon_energy.html#pvlib.spectrum.average_photon_energy). Note that since we have more than one spectral irradiance distribution, we need to use a ``pandas.DataFrame`` as an input ot the above function.

In [ ]:
spectra = pd.DataFrame(spectral_poa).T  # convert to dataframe and transpose
spectra.index = time_labels  # add time index
spectra.columns = wavelength  # add wavelength column headers

ape = spectrum.average_photon_energy(spectra)
ape

Note that the units of the APE are electronvolts (eV). What do you notice for the different spectra? What values do you expect to get for a red- and a blue-shifted spectrum?

## Step 6: Investiate dfferent atmospheric optical parameters

Repeat the previous steps using different atmospheric parameters (i.e., ``water_vapor_content``, ``tau500``, ``ozone``). How do these affect the spectrum?